In [ ]:
import sqlalchemy as db
from urllib.parse import quote_plus
import pandas as pd
import numpy as np
#Database credentials
user = 'user'
password = 'pass'
if "@" in password:
    password = quote_plus(password)
host = 'host'
port = 'port'
#database = 'db?useSSL=false'
database = 'db'
engine = None
#Connector to db
if "?useSSL=false" in database:
    database = database.split('?')[0]
    engine = db.create_engine('mysql+mysqlconnector://' + user + ':' + password + '@' + host + ':' + port + '/' + database, connect_args={'ssl_disabled': 'True'})
else:
    engine = db.create_engine('mysql+pymysql://' + user + ':' + password + '@' + host + ':' + port + '/' + database)

In [ ]:
teammates = pd.read_csv('builders.csv')
teammates.head(5)

In [ ]:
num_teammates = teammates.count(axis=1)
num_teammates

In [ ]:
#Get table worlds in db has columns (row id, world id, and world name)
meta = db.MetaData()
db.MetaData.reflect(meta,bind=engine)
worlds = meta.tables['co_world']

In [ ]:
print(worlds.columns)

In [ ]:
#Query to retrieve world id and world name
query = db.select([
    worlds.c.id,
    worlds.c.world,
])

result = engine.execute(query).fetchall()
result

In [ ]:
#Make list of worlds of interest to count blocks
build_worlds = teammates["world"].unique()
world_coding = []
for world in build_worlds:
    for entry in result:
        if entry[1] ==  world:
            world_coding.append(entry)
            break
world_coding

In [ ]:
#Get mapping of block type id to block name
material_map = meta.tables['co_material_map']
material_query = db.select([
    material_map.c.id,
    material_map.c.material,
])
material_result = engine.execute(material_query).fetchall()
block_types = []
for material in material_result:
    block_types.append(material[1])
block_counts = pd.DataFrame(columns = block_types)
block_counts

In [ ]:
for i in range(len(teammates)):
    row = teammates.iloc[[i]]
    group_size = num_teammates.iloc[[i]][i]-2
    world = row["world"][i]
    world_id = -1
    for w in world_coding:
        if w[1] == world:
            world_id = w[0]
            #print(w[1])
            break
    #Skip world if whitefield2 = empty world
    if world_id == 88:
        continue
    #Get blocks from world of interest
    blocks = meta.tables['co_block']
    users = meta.tables['co_user']
    block_result = []
    if group_size == 1:
        query_users = db.select([
            users.c.rowid,
            users.c.user,
        ]).where((users.c.user == row['builder1'][i]))
        user_result = engine.execute(query_users).fetchall()
        #print(user_result)
        query_blocks = db.select([
        blocks.c.type,
        ]).where((blocks.c.wid == world_id) & (blocks.c.user == user_result[0][0]))
        block_result = engine.execute(query_blocks).fetchall()
    elif group_size == 2:
        query_users = db.select([
            users.c.rowid,
            users.c.user,
        ]).where((users.c.user == row['builder1'][i]) | (users.c.user == row['builder2'][i]))
        user_result = engine.execute(query_users).fetchall()
        #print(user_result)
        query_blocks = db.select([
        blocks.c.type,
        ]).where((blocks.c.wid == world_id) & ((blocks.c.user == user_result[0][0])|(blocks.c.user == user_result[1][0])))
        block_result = engine.execute(query_blocks).fetchall()
    elif group_size == 3:
        query_users = db.select([
            users.c.rowid,
            users.c.user,
        ]).where((users.c.user == row['builder1'][i]) | (users.c.user == row['builder2'][i]) | (users.c.user == row['builder3'][i]))
        user_result = engine.execute(query_users).fetchall()
        #print(user_result)
        query_blocks = db.select([
        blocks.c.type,
        ]).where((blocks.c.wid == world_id) & ((blocks.c.user == user_result[0][0])|(blocks.c.user == user_result[1][0])|(blocks.c.user == user_result[2][0])))
        block_result = engine.execute(query_blocks).fetchall()
    elif group_size == 4:
        query_users = db.select([
            users.c.rowid,
            users.c.user,
        ]).where((users.c.user == row['builder1'][i]) | (users.c.user == row['builder2'][i]) | (users.c.user == row['builder3'][i]) | (users.c.user == row['builder4'][i]))
        user_result = engine.execute(query_users).fetchall()
        #print(user_result)
        query_blocks = db.select([
        blocks.c.type,
        ]).where((blocks.c.wid == world_id) & ((blocks.c.user == user_result[0][0])|(blocks.c.user == user_result[1][0])|(blocks.c.user == user_result[2][0])|(blocks.c.user == user_result[3][0])))
        block_result = engine.execute(query_blocks).fetchall()
    elif group_size == 5:
        query_users = db.select([
            users.c.rowid,
            users.c.user,
        ]).where((users.c.user == row['builder1'][i]) | (users.c.user == row['builder2'][i]) | (users.c.user == row['builder3'][i]) | (users.c.user == row['builder4'][i]) | (users.c.user == row['builder5'][i]))
        user_result = engine.execute(query_users).fetchall()
        #print(user_result)
        query_blocks = db.select([
        blocks.c.type,
        ]).where((blocks.c.wid == world_id) & ((blocks.c.user == user_result[0][0])|(blocks.c.user == user_result[1][0])|(blocks.c.user == user_result[2][0])|(blocks.c.user == user_result[3][0])|(blocks.c.user == user_result[4][0])))
        block_result = engine.execute(query_blocks).fetchall()
    unique, counts = np.unique(block_result, return_counts=True)
    block_usages = dict(zip(unique, counts))
    for j, k in block_usages.items():
        material = material_result[j-1][1]
        block_counts.at[row["id"][i],material] = k
    print("Base " + str(row["id"][i]) + "/" + str(len(teammates)) + " Completed!")

In [ ]:
block_counts

In [ ]:
#Whitefield2023, Caribou, Whitefield, Goldcrown, Mahometafterschool, sdp1, sdp2, uncc2023, sdp3, sdp4, sdp5
#Have Whitefield2, Caribou, Goldcrown, sdp1, 
blocks = meta.tables['co_block']
users = meta.tables['co_user']
query_users = db.select([
            users.c.rowid,
            users.c.user,
        ]).where((users.c.user == "MCSdolphin") | (users.c.user == "MCSbear") | (users.c.user == "MCSeagle"))
user_result = engine.execute(query_users).fetchall()
        #print(user_result)
query_blocks = db.select([
        blocks.c.type,
        ]).where((blocks.c.wid == 92) & ((blocks.c.user == user_result[0][0])|(blocks.c.user == user_result[1][0])|(blocks.c.user == user_result[2][0])))
block_result = engine.execute(query_blocks).fetchall()
block_result

In [ ]:
#Create csv for Block_Counter to use
block_counts.to_csv('block_counts.csv',index=True)